O objetivo do código aqui é determinar:

1. O nível de lag a ser utilizado nos modelos (usando RF de base)

2. Se o RF deve usar séries estacionárias ou não

In [32]:
from sklearn.ensemble import RandomForestRegressor
from sktime.forecasting.compose import make_reduction
from sktime.split import temporal_train_test_split, ExpandingWindowSplitter, SlidingWindowSplitter
from sktime.forecasting.model_evaluation import evaluate
from sktime.forecasting.base import ForecastingHorizon
from sktime.utils.plotting import plot_series
import pandas as pd
import utils

In [21]:
# Importa dados e remove dados desnecessários
df = pd.read_csv("dados/lags.csv", sep=";", decimal=",", thousands=".")
df = utils.index_to_period(df)
df = df.dropna()
df = df.drop(['2003-09', '2003-10', '2003-11', '2003-12'])
df

,ipca,ipca_15,selic,m1,m2,m3,m4,ibovespa,cambio,cambio_real,...,crb_food_3,crb_food_4,crb_food_5,crb_food_6,crb_metal_1,crb_metal_2,crb_metal_3,crb_metal_4,crb_metal_5,crb_metal_6
month,,,,,,,,,,,,,,,,,,,,,
2004-01,0.76,0.68,16.500000,101206857,408357097,850484302,968637026,21851.44,2.8518,88.177,...,254.36,250.82,237.50,231.97,276.69,249.27,246.28,229.08,213.90,215.95
2004-02,0.61,0.90,16.500000,102857652,411584365,860505531,981337578,21755.02,2.9303,89.876,...,256.65,254.36,250.82,237.50,291.79,276.69,249.27,246.28,229.08,213.90
2004-03,0.47,0.40,16.387097,101282219,413528931,870826987,989838137,22142.26,2.9055,87.099,...,250.24,256.65,254.36,250.82,316.06,291.79,276.69,249.27,246.28,229.08
2004-04,0.37,0.21,16.116667,101821601,415095168,876412419,995559726,19607.23,2.9060,85.537,...,268.94,250.24,256.65,254.36,327.01,316.06,291.79,276.69,249.27,246.28
2004-05,0.51,0.54,16.000000,103074482,429712981,908888294,1012425016,19544.67,3.1004,89.750,...,283.68,268.94,250.24,256.65,315.88,327.01,316.06,291.79,276.69,249.27
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-05,0.23,0.51,13.750000,577826684,5322265166,10022339920,11110115291,108335.07,4.9828,53.920,...,527.16,521.95,533.35,557.74,1056.98,1088.08,1067.57,1120.19,1011.58,980.30
2023-06,-0.08,0.04,13.750000,588295220,5423979831,10218926744,11238752433,118087.00,4.8516,53.000,...,533.88,527.16,521.95,533.35,987.00,1056.98,1088.08,1067.57,1120.19,1011.58
2023-07,0.12,-0.07,13.750000,617150371,5502967358,10359936160,11332427031,121942.98,4.8008,53.410,...,528.03,533.88,527.16,521.95,992.69,987.00,1056.98,1088.08,1067.57,1120.19


In [22]:
y = df['ipca']
X = df.drop(columns=['ipca'])
y_train, y_test, X_train, X_test = temporal_train_test_split(y, X, test_size=79)

In [23]:
# Limita seleção de parâmetros a sqrt(p), conforme ISL
regressor = RandomForestRegressor(max_features="sqrt")
fh = ForecastingHorizon(1, is_relative=True)
forecaster = make_reduction(regressor)
forecaster.fit(y_train, X_train, fh=fh)

RecursiveTabularRegressionForecaster(estimator=RandomForestRegressor(max_features='sqrt'))

In [28]:
cv = ExpandingWindowSplitter(fh=fh, initial_window=y_train.size)
eval_df = evaluate(forecaster, cv, y, X, strategy="refit", return_data=True)

In [56]:
# Transformações necessárias, pois 'evaluate' retorna 'y_pred' como série de séries
# numa coluna do DataFrame
y_pred = pd.Series([value[0] for _, value in eval_df['y_pred'].items()])
y_pred.index = y_test.index
y_pred

C:\Users\asfra\AppData\Local\Temp\ipykernel_10488\537483885.py:3: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  y_pred = pd.Series([value[0] for _, value in eval_df['y_pred'].items()])


2017-03    0.4607
2017-04    0.4193
2017-05    0.3230
2017-06    0.2844
2017-07    0.1436
            ...  
2023-05    0.4015
2023-06    0.3755
2023-07    0.2272
2023-08    0.2099
2023-09    0.3194
Freq: M, Length: 79, dtype: float64

In [34]:
labels = ['y_train', 'y_test', 'y_pred']
plot_series(y_train, y_test, y_pred, labels=labels)

TypeError: No valid mtype could be identified for object of type <class 'pandas.core.series.Series'>. Errors returned are as follows, in format [mtype]: [error message] 
pd.DataFrame: obj must be a pandas.DataFrame, found <class 'pandas.core.series.Series'>
pd.Series: obj should not be of 'object' dtype
np.ndarray: obj must be a numpy.ndarray, found <class 'pandas.core.series.Series'>
